In [2]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab2 import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install.
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 52.3 MB/s 
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 59 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155225 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u312-b07-0ubuntu1~18.04) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.

In [4]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [5]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

copy some wiki data

In [6]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
auth.authenticate_user()

In [7]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth

project_id = 'core-period-321814'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

Copying gs://wikidata_preprocessed/multistream1_preprocessed.parquet...
/ [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [28]:
from pathlib import Path 
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
doc_title_pairs = parquetFile.limit(1000).select("title", "id").rdd
d_d_t_pairs = doc_title_pairs.map(lambda x: (x[1], (x[1], x[0])))
# create dict of (id, title)
doc_title_dict = d_d_t_pairs.collectAsMap()
print(doc_title_dict)

{12: (12, 'Anarchism'), 25: (25, 'Autism'), 39: (39, 'Albedo'), 290: (290, 'A'), 303: (303, 'Alabama'), 305: (305, 'Achilles'), 307: (307, 'Abraham Lincoln'), 308: (308, 'Aristotle'), 309: (309, 'An American in Paris'), 316: (316, 'Academy Award for Best Production Design'), 324: (324, 'Academy Awards'), 330: (330, 'Actrius'), 332: (332, 'Animalia (book)'), 334: (334, 'International Atomic Time'), 336: (336, 'Altruism'), 339: (339, 'Ayn Rand'), 340: (340, 'Alain Connes'), 344: (344, 'Allan Dwan'), 358: (358, 'Algeria'), 359: (359, 'List of Atlas Shrugged characters'), 569: (569, 'Anthropology'), 572: (572, 'Agricultural science'), 573: (573, 'Alchemy'), 579: (579, 'Alien'), 580: (580, 'Astronomer'), 586: (586, 'ASCII'), 590: (590, 'Austin (disambiguation)'), 593: (593, 'Animation'), 594: (594, 'Apollo'), 595: (595, 'Andre Agassi'), 597: (597, 'Austroasiatic languages'), 599: (599, 'Afroasiatic languages'), 600: (600, 'Andorra'), 612: (612, 'Arithmetic mean'), 615: (615, 'American Footb

term frequancy

below is a function that calculates tf for a single text

In [29]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  ret_list = []
  counter = Counter(tokens)
  for token in counter:
    if token not in all_stopwords:
        ret_list.append((token, (id, counter[token])))
  return ret_list

In [30]:
wc_title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))

In [31]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return sorted(unsorted_pl)

In [32]:
postings_for_title = wc_title.groupByKey().mapValues(reduce_word_counts)
print(postings_for_title.take(50))

[('anarchism', [(12, 1)]), ('autism', [(25, 1)]), ('albedo', [(39, 1)]), ('alabama', [(303, 1), (1285, 1), (1286, 1), (1484, 1)]), ('achilles', [(305, 1), (2421, 1)]), ('abraham', [(307, 1), (1436, 1)]), ('lincoln', [(307, 1)]), ('aristotle', [(308, 1)]), ('american', [(309, 1), (615, 1), (659, 1), (771, 1), (863, 1), (869, 1), (966, 1), (1110, 1), (1111, 1), (1112, 1), (1201, 1), (1241, 1), (1488, 1), (1558, 1), (1890, 1), (1902, 1), (1950, 1), (1973, 1), (2011, 1), (2175, 1), (2357, 1), (2386, 1)]), ('paris', [(309, 1)]), ('academy', [(316, 1), (324, 1)]), ('award', [(316, 1)]), ('best', [(316, 1)]), ('production', [(316, 1)]), ('design', [(316, 1)]), ('awards', [(324, 1)]), ('actrius', [(330, 1)]), ('animalia', [(332, 1)]), ('book', [(332, 1), (1726, 1)]), ('international', [(334, 1)]), ('atomic', [(334, 1), (673, 1), (1194, 1), (1200, 1), (1206, 1), (2014, 1)]), ('time', [(334, 1), (1638, 1)]), ('altruism', [(336, 1)]), ('ayn', [(339, 1)]), ('rand', [(339, 1)]), ('alain', [(340, 1)

In [33]:
# pft_filtered = postings_for_title.filter(lambda x: len(x[1])>1)
# pft_filtered = postings_for_title

In [34]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  ret = postings.map(lambda x: (x[0], len(x[1])))
  print(ret.take(5))
  return ret

In [35]:
df_for_title = calculate_df(postings_for_title)

[('anarchism', 1), ('autism', 1), ('albedo', 1), ('alabama', 4), ('achilles', 2)]


In [36]:
df_for_title_dict = df_for_title.collectAsMap()

In [37]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  print(postings.take(2))
  partitions = postings.groupBy(lambda x : token2bucket_id(x[0]))
  mapp = partitions.map(lambda x : InvertedIndex.write_a_posting_list(x))
  return mapp

In [38]:
posting_locs_list_title = partition_postings_and_write(postings_for_title).collect()

[('anarchism', [(12, 1)]), ('autism', [(25, 1)])]


In [39]:
super_posting_locs_title = defaultdict(list)
for posting_loc in posting_locs_list_title:
  for k, v in posting_loc.items():
    super_posting_locs_title[k].extend(v)

In [40]:
# Create inverted index instance
inverted_title = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_title.posting_locs = super_posting_locs_title
# Add the token - df dictionary to the inverted index
inverted_title.df = df_for_title_dict
# Add the id - title dictionary to the inverted index
inverted_title.wikiid_title = doc_title_dict
# write the global stats out
inverted_title.write_index('.', 'title_index')

In [41]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    x = inverted.df[w]
    print(x)
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      # print(doc_id)
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      # print(tf)
      posting_list.append((doc_id, tf))
    return posting_list

In [42]:
## test the final index
pl2 = read_posting_list(inverted_title, 'american')
print(pl2)

22
[(309, 1), (615, 1), (659, 1), (771, 1), (863, 1), (869, 1), (966, 1), (1110, 1), (1111, 1), (1112, 1), (1201, 1), (1241, 1), (1488, 1), (1558, 1), (1890, 1), (1902, 1), (1950, 1), (1973, 1), (2011, 1), (2175, 1), (2357, 1), (2386, 1)]


In [43]:
# returnes list of tuples : (token, tf_in_q)
def tokenize_query(query):
  tokens = [token.group() for token in RE_WORD.finditer(query.lower())]
  # YOUR CODE HERE
  ret_list = []
  counter = Counter(tokens)
  for token in counter:
    if token not in all_stopwords:
        ret_list.append((token,counter[token]))
  return ret_list

In [49]:
def search_title(query):
  tokenized_q = tokenize_query(query)
  counter = Counter()
  ret_list = []
  for t in tokenized_q: 
    pl = read_posting_list(inverted_title, t[0])
    ret = map(lambda x: x[0], pl)
    counter.update(ret)
  sorted_counter = counter.most_common()
  print(sorted_counter)
  for pair in sorted_counter:
    ret_list.append(inverted_title.wikiid_title[pair[0]])
  return ret_list

search_title("Academy Award for")



2
1
[(316, 2), (324, 1)]


[(316, 'Academy Award for Best Production Design'), (324, 'Academy Awards')]